In [4]:
import os
import glob
import numpy as np
from skimage import io, color, transform, feature
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


In [ ]:

# Paramètres pour le prétraitement
IMAGE_SIZE = (64, 64)  
PIXELS_PER_CELL = (8, 8)
CELLS_PER_BLOCK = (2, 2)
ORIENTATIONS = 9



In [ ]:
def extraire_hog(image):
    """
    Convertit une image en niveaux de gris, la redimensionne,
    et extrait les caractéristiques HOG.
    """
    if len(image.shape) == 3:
        image = color.rgb2gray(image)
    # Redimensionner l'image
    image_resized = transform.resize(image, IMAGE_SIZE, anti_aliasing=True)
    hog_features = feature.hog(image_resized,
                               orientations=ORIENTATIONS,
                               pixels_per_cell=PIXELS_PER_CELL,
                               cells_per_block=CELLS_PER_BLOCK,
                               block_norm='L2-Hys')
    return hog_features



In [ ]:
def charger_donnees(data_dir):
    """
    Parcourt les dossiers de la structure donnée et retourne
    deux tableaux : X (les caractéristiques extraites) et y (les labels).
    """
    X = []
    y = []
    for label in os.listdir(data_dir):
        label_path = os.path.join(data_dir, label)
        if os.path.isdir(label_path):
            count = 0  
            for image_path in glob.glob(os.path.join(label_path, "*.jpg")):
                if count >= 2000:
                    break  
                try:
                    image = io.imread(image_path)
                    features = extraire_hog(image)
                    X.append(features)
                    y.append(label)
                    count += 1
                except Exception as e:
                    print(f"Erreur lors du traitement de {image_path}: {e}")
    return np.array(X), np.array(y)


In [8]:
# Charger les données d'entraînement et de validation
train_dir = os.path.join("images", "train")
validation_dir = os.path.join("images", "validation")

print("Chargement des données d'entraînement...")
X_train, y_train = charger_donnees(train_dir)
print(f"{len(X_train)} images d'entraînement chargées.")

print("Chargement des données de validation...")
X_val, y_val = charger_donnees(validation_dir)
print(f"{len(X_val)} images de validation chargées.")
 

Chargement des données d'entraînement...
12436 images d'entraînement chargées.
Chargement des données de validation...
7066 images de validation chargées.


In [ ]:

# Création d'un pipeline de traitement incluant la normalisation etle classifieur SVM
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(kernel='linear', probability=True))
])

# Entraînement du modèle
print("Entraînement du modèle SVM...")
pipeline.fit(X_train, y_train)

print("Évaluation du modèle sur le jeu de validation...")
y_pred = pipeline.predict(X_val)
rapport = classification_report(y_val, y_pred)
print(rapport)


Entraînement du modèle SVM...
Évaluation du modèle sur le jeu de validation...
              precision    recall  f1-score   support

       angry       0.25      0.37      0.29       960
     disgust       0.19      0.48      0.27       111
        fear       0.26      0.30      0.28      1018
       happy       0.60      0.46      0.52      1825
     neutral       0.35      0.31      0.33      1216
         sad       0.30      0.24      0.27      1139
    surprise       0.50      0.49      0.50       797

    accuracy                           0.37      7066
   macro avg       0.35      0.38      0.35      7066
weighted avg       0.39      0.37      0.37      7066



In [13]:
# save model
import pickle

with open('model.pkl', 'wb') as model_file:
    pickle.dump(pipeline, model_file)
print("Model saved to model.pkl")


Model saved to model.pkl


In [9]:
# load model
import pickle

 


with open('model.pkl', 'rb') as model_file:
    model = pickle.load(model_file)
    print("Model loaded from model.pkl")
    y_pred = model.predict(X_val)
    rapport = classification_report(y_val, y_pred)
    print(rapport)


Model loaded from model.pkl
              precision    recall  f1-score   support

       angry       0.25      0.37      0.29       960
     disgust       0.19      0.48      0.27       111
        fear       0.26      0.30      0.28      1018
       happy       0.60      0.46      0.52      1825
     neutral       0.35      0.31      0.33      1216
         sad       0.30      0.24      0.27      1139
    surprise       0.50      0.49      0.50       797

    accuracy                           0.37      7066
   macro avg       0.35      0.38      0.35      7066
weighted avg       0.39      0.37      0.37      7066



In [14]:
model

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm', SVC(kernel='linear', probability=True))])

In [ ]:
# savve on mlflow
import mlflow

mlflow.set_tracking_uri("https://mlflow-cloud-g1-1d0d7b4ea267.herokuapp.com/")
experiment = mlflow.set_experiment("facial_emotions_classification")
os.environ['AWS_ACCESS_KEY_ID'] = ""
os.environ['AWS_SECRET_ACCESS_KEY'] = ""

with mlflow.start_run(experiment_id=experiment.experiment_id):
    mlflow.sklearn.log_model(model, "model",registered_model_name="emotion_recognition")
    mlflow.log_metric("accuracy", 0.75)
    mlflow.log_param("data_dir", "images")
    mlflow.log_param("train_size", len(X_train))
    mlflow.log_param("val_size", len(X_val))

print("Model saved to MLflow")

2025/03/03 11:08:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run enthused-croc-741 at: https://mlflow-cloud-g1-1d0d7b4ea267.herokuapp.com/#/experiments/130960509290812118/runs/e745d1e870d84aac803f57dde877a1de
🧪 View experiment at: https://mlflow-cloud-g1-1d0d7b4ea267.herokuapp.com/#/experiments/130960509290812118


MlflowException: API request to https://mlflow-cloud-g1-1d0d7b4ea267.herokuapp.com/api/2.0/mlflow-artifacts/artifacts/130960509290812118/e745d1e870d84aac803f57dde877a1de/artifacts/model/model.pkl failed with exception HTTPSConnectionPool(host='mlflow-cloud-g1-1d0d7b4ea267.herokuapp.com', port=443): Max retries exceeded with url: /api/2.0/mlflow-artifacts/artifacts/130960509290812118/e745d1e870d84aac803f57dde877a1de/artifacts/model/model.pkl (Caused by ProtocolError('Connection aborted.', ConnectionAbortedError(10053, 'Une connexion établie a été abandonnée par un logiciel de votre ordinateur hôte', None, 10053, None)))